In [ ]:
import requests
import os
import json
import preprocessor as p #tweetpreprocessor --> https://pypi.org/project/tweet-preprocessor/
from langdetect import detect #https://pypi.org/project/langdetect/
from csv import writer

from ernie import SentenceClassifier #Model importieren
import numpy as np

from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re
import bs4 as bs
from urllib.request import urlopen
import pandas as pd
import pandas_datareader.data as pdr
from datetime import datetime


BEARER_TOKEN=""
classifier = SentenceClassifier(model_path='output')

in_position = False
TRADE_SYMBOL = "BTCUSDT"
TRADE_QUANTITY = 0.005
c=0
sentimentLst = []
in_position = False


def getIndex(): #WebScraper für den Fear & Greed Index
    cnn = "https://money.cnn.com/data/fear-and-greed/"
    req = Request(url=cnn, headers={'user-agent': 'my-app/0.0.1'})
    response = urlopen(req)
    # response
    feargreedindex = {}
    html = BeautifulSoup(response)

    feargreedindex = html.find(id='needleChart')

    dataRows = feargreedindex.findAll('li')
    indexString = dataRows[0]
    indexString = re.findall(r'[0-9]+', str(indexString))
    return float(indexString[0])

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    return response.json()


def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )


def set_rules(headers, delete, bearer_token): #Rules an API senden, um Daten zu empfangen
    #Rules können angepasst werden
    sample_rules = [
        {"value": "#bitcoin", "tag": "bitcoin"},
        # {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )


def get_stream(headers, set, bearer_token):
    global sentiments,fg_index_d,c
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            tweet = json_response['data']['text']
            tweet = p.clean(tweet)
            tweet = tweet.replace(':','')
            index = getIndex()

            try:
                if detect(tweet) == 'en':
                    try: 
                        #-1 BEARISH 0 NEUTRAL 1 BULLISH
                        classes = ["Bearish", "Neutral", "Bullish"]
                        probabilities = classifier.predict_one(tweet)
                        polarity = classes[np.argmax(probabilities)]
                        if c<=15:
                            print('')
                            print(f'{tweet}******** Predicted Sentiment: {polarity} ********')
                            print('')
                            c+=1
                        if polarity=='Neutral':
                            sentiments['Neutral']+=1
                        elif polarity=='Bullish':
                            sentiments['Bullish']+=1
                        elif polarity=='Bearish':
                            sentiments['Bearish']+=1
                        sentimentLst.append(polarity)
                        print(f"******** Live Bullish Tweet Count : {sentiments['Bullish']} ********",end='\r')
                        index = getIndex()
                        if index<=49 and index>=0:
                            fg_index_d['Bearish']+=1
                        elif index>=50 and index<=60:
                            fg_index_d['Neutral']+=1
                        elif index>60:
                            fg_index_d['Bullish']+=1
                        if sentiments['Bullish']==35:
                              print(sentiments['Bullish'])
                              break
                        if len(sentimentLst) > 50:
                            endList = sentimentLst[-50:]
                            # print("******** TOTAL BULLISH: " + str(endList.count('Bullish')))
                            # print("******** TOTAL BEARISH: " + str(endList.count('Bearish')))

                            if endList.count('Bullish') > 40 and index < 40:
                                #BUY
                                if in_position:
                                    print("******** BUY ******** but we own")
                                else: 
                                    print("******** BUY ********")
                                    # order_succeeded = order(SIDE_BUY, TRADE_QUANTITY, TRADE_SYMBOL)
                                    if order_succeeded:
                                        in_position = True
                            elif endList.count('Bearish') > 40 and index > 60:
                                #SELL
                                if in_position:
                                    # order_succeeded = order(SIDE_SELL, TRADE_QUANTITY, TRADE_SYMBOL)
                                    if order_succeded:
                                        in_position = False
                                else:
                                    print('******** SELL ******** but we dont own')
                    except:
                        pass
                    # tweetLst = [tweet]

                    # with open('bitcoindata.csv', 'a+', newline='') as write_obj:
                    #     csv_writer = writer(write_obj)
                    #     csv_writer.writerow(tweetLst)
            except:
                pass
        
sentiments={'Bullish':0,'Bearish':0,'Neutral':0}
fg_index_d={'Bullish':0,'Bearish':0,'Neutral':0}
def main():
    global sentiments,fg_index_d
    try:
        bearer_token = BEARER_TOKEN
        headers = create_headers(bearer_token)
        rules = get_rules(headers, bearer_token)
        delete = delete_all_rules(headers, bearer_token, rules)
        set = set_rules(headers, delete, bearer_token)
        get_stream(headers, set, bearer_token)
    except:
        pass
    print(f"********Current Number of Bullish Tweets: {sentiments['Bullish']}********")
    print(f"******** CNN Fear and Greed Index: {getIndex()} ********")
    
          
      
while True: #News Scraper + Sentiment Analysis 
    main()
    crypto_name='BTC'
    source=urlopen(f'https://news.google.com/search?q={crypto_name}?h%3Den-US&gl=US&ceid=US:en&hl=en-US').read()
    soup = bs.BeautifulSoup(source,'lxml')
    import pysentiment2 as ps
    lm = ps.LM()
    sentiment_d={'Positive':0,'Negative':0}
    lnks=soup.find_all('h3')
    for itert in lnks:
        lnk=itert.find('a').text
        tokens = lm.tokenize(lnk)
        score = lm.get_score(tokens)
        if score['Positive']>0:
            sentiment_d['Positive']+=1
        if score['Negative']>0:
            sentiment_d['Negative']+=1
    
    print(f'******** Total Number of News Crawled: {sum(sentiment_d.values())} ********')
    print(f'******** Sentiment of News: {max(sentiment_d, key=sentiment_d.get)} ********')
    end=datetime.today()
    BTC = pdr.DataReader('BTC-USD','yahoo',datetime(end.year-1,end.month,end.day),end)
    ShortEMA = BTC.Close.ewm(span=12, adjust=False).mean()
    #EMA ausrechnen
    LongEMA = BTC.Close.ewm(span=26, adjust=False).mean()
    #MACD ausrechnen
    MACD = ShortEMA - LongEMA
    #Linie des MACD ausrechnen
    signal = MACD.ewm(span=9, adjust=False).mean() 
    BTC['MACD'] = MACD
    BTC['Signal_Line'] = signal
    macd_signal='Buy'
    if BTC.iloc[-1]['MACD']<0 :
        macd_signal='Sell'
    print(f'******** Current MACD: {macd_signal} ********')
    def computeRSI (data, time_window):
        diff = data.diff(1).dropna()      

     
        up_chg = 0 * diff
        down_chg = 0 * diff


        up_chg[diff > 0] = diff[ diff>0 ]

 
        down_chg[diff < 0] = diff[ diff < 0 ]

        #EMA's kalkulieren
        up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
        down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()

        rs = abs(up_chg_avg/down_chg_avg)
        rsi = 100 - 100/(1+rs)
        return(rsi)

    rsi_list=[]
    for g in computeRSI(BTC['Close'], 14):
        if str(g)!='nan':
            rsi_list.append(g)

    rsi_indicator=''
    if rsi_list[-1]>70:
        rsi_indicator='Sell'
    elif rsi_list[-1]<30:
        rsi_indicator='Buy'
    else:
        rsi_indicator='Neutral'
    print(f'******** Current RSI: {rsi_indicator} ********')
    tweet_score=(sentiments['Bullish']/sum(sentiments.values()))*100
    news_score=(sentiment_d['Positive']/sum(sentiment_d.values()))*100
    fg_score=(fg_index_d['Bullish']/sum(fg_index_d.values()))*100

    tweet_weightage=30
    news_weightage=40
    fg_score_weightage=30

    score=tweet_score*(tweet_weightage/100)+news_score*(news_weightage/100)+fg_score*(fg_score_weightage/100)
    if score>80: #Score anpassen
        print(f'******** CURRENT OVERALL SCORE : {score} ***** Waiting to set the Buy Order..')
    else:
        print(f'******** CURRENT OVERALL SCORE : {score} *****')

    try:
        from binance.enums import *
        from binance.client import Client
        import binance

        if score > 80:
            client = Client('','')
            #Order Function mit Kauf- und Verkaufsfunktion bei 5% Profit  


            stop_loss=float(client.get_avg_price(symbol='BTCUSDT')['price']) - (float(client.get_avg_price(symbol='BTCUSDT')['price'])*0.05)
            avg_price = client.get_avg_price(symbol='BTCUSDT')['price']
            print(stop_loss,avg_price)

            order = client.order_market_buy(symbol='BTCUSDT',quantity=0.001)
            print(f"******** BTC BOUGHT AT : {avg_price} WITH STOP LOSS AT :{stop_loss} ******* Waiting for Sell")

            while True:
              current_price = client.get_avg_price(symbol='BTCUSDT')['price']
              if float(current_price)>float(stop_loss) and float(current_price) < (float(avg_price) + (float(avg_price)*0.005)):
                print(f"Price : {client.get_avg_price(symbol='BTCUSDT')['price']} Stop Loss {stop_loss}")
                pass
              else:
                order = client.order_market_sell(symbol='BTCUSDT',quantity=0.001)
                if stop_loss>current_price:
                      print('***** Stop Loss Hit ***** Sold')
                elif float(current_price) >= (float(avg_price) + (float(avg_price)*0.005)):
                      print(f'****** BTC SOLD *** Your profit is {current_price-avg_price} USD ****')
                break
    except:
         pass
    c=0
    sentiments={'Bullish':0,'Bearish':0,'Neutral':0}
    fg_index_d={'Bullish':0,'Bearish':0,'Neutral':0}